# Comparative experiment with Attention Model for Solving p-Median

## Prepare: Install dependencies
### Install with pip
* python=3.7
* PyTorch>=1.1
* numpy
* tqdm
* cv2
* tensorboard_logger

In [1]:
from utils import torch_load_cpu, load_problem, get_inner_model, move_to
from nets.attention_model import AttentionModel
from tensorboard_logger import Logger as TbLogger
import torch

## load the settings

In [2]:
# load the run args
%run options

# Set the random seed
torch.manual_seed(1234)

# Optionally configure tensorboard
tb_logger = None
if not opts.no_tensorboard:
    tb_logger = TbLogger(os.path.join(opts.log_dir, "{}_{}".format(opts.problem, opts.n_users, opts.n_facilities), opts.run_name))

# Set the device
use_cuda=True
opts.device = torch.device("cuda" if use_cuda else "cpu")
opts

Namespace(baseline=None, batch_size=10, bl_alpha=0.05, bl_warmup_epochs=0, checkpoint_encoder=False, checkpoint_epochs=1, data_distribution=None, device=device(type='cuda'), embedding_dim=128, epoch_size=50, epoch_start=0, eval_batch_size=10, eval_only=False, exp_beta=0.8, hidden_dim=128, load_path=None, log_dir='logs', log_step=50, lr_critic=0.0001, lr_decay=1, lr_model=0.0001, max_grad_norm=1.0, model='attention', n_encode_layers=3, n_epochs=100, n_facilities=20, n_users=50, no_cuda=False, no_progress_bar=False, no_tensorboard=False, normalization='batch', output_dir='outputs', p=8, problem='PM', r=0.1, resume=None, run_name='20_8_20240903T152832', save_dir='outputs\\PM\\20_8_20240903T152832', seed=2023, shrink_size=None, tanh_clipping=10.0, use_cuda=True, val_dataset='./data/Test50_20_8.pkl', val_size=10)

## Figure out what's the problem

In [3]:
problem = load_problem(opts.problem)
problem

problems.PM.problem_PM.PM

## Initialize our policy network

In [4]:
model_class = {
    # 'pointer': PointerNetwork,
    'attention': AttentionModel
}.get(opts.model, None)

assert model_class is not None, "Unknown model: {}".format(model_class)
model = model_class(
    opts.embedding_dim,
    opts.hidden_dim,
    problem,
    n_encode_layers=opts.n_encode_layers,
    mask_inner=True,
    mask_logits=True,
    normalization=opts.normalization,
    tanh_clipping=opts.tanh_clipping,
    checkpoint_encoder=opts.checkpoint_encoder,
    shrink_size=opts.shrink_size,
    dy=False
).to(opts.device)

model

AttentionModel(
  (init_embed): Linear(in_features=2, out_features=128, bias=True)
  (init_dynamic): Linear(in_features=1, out_features=32, bias=True)
  (l2_dynamic): Linear(in_features=32, out_features=64, bias=True)
  (l3_dynamic): Linear(in_features=64, out_features=128, bias=True)
  (embedder): GraphAttentionEncoder(
    (layers): Sequential(
      (0): MultiHeadAttentionLayer(
        (0): SkipConnection(
          (module): MultiHeadAttention()
        )
        (1): Normalization(
          (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): SkipConnection(
          (module): Sequential(
            (0): Linear(in_features=128, out_features=512, bias=True)
            (1): ReLU()
            (2): Linear(in_features=512, out_features=128, bias=True)
          )
        )
        (3): Normalization(
          (normalizer): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
  

## load the AM model

In [5]:
opts.load_path = './output/epoch-99.pt'
# load model from load_path
assert opts.load_path is None or opts.resume is None, "Only one of load path and resume can be given"
load_path = opts.load_path if opts.load_path is not None else opts.resume
if load_path is not None:
    print('  [*] Loading the trained model from {}'.format(load_path))
    load_data = torch_load_cpu(load_path)

# Overwrite model parameters by parameters to load q
model_ = get_inner_model(model)
model.load_state_dict({**model_.state_dict(), **load_data.get('model', {})})

  [*] Loading the trained model from ./output/epoch-99.pt


<All keys matched successfully>

## Load the real-world datasets

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### LandScan Population Distribution
LandScan data is preprocessed and excluded the regions with no night-time population. Each point in this dataset represents the population aggregated to the centroid of the corresponding grid cell.

In [7]:
%%time
ls = gpd.read_file("./data/real/haidian_community_pro.shp")
ls['POINT_X'] = ls.geometry.x
ls['POINT_Y'] = ls.geometry.y
ls.head(3)

Wall time: 175 ms


,经度84,纬度84,POINT_X,POINT_Y,NEAR_FID,NEAR_DIST,NEAR_X,NEAR_Y,geometry
0,116.268247,39.970932,950178.402354,4.439620e+06,53,822.227299,950742.699785,4.439022e+06,POINT (950178.402 4439619.552)
1,116.278045,39.965943,951048.965144,4.439115e+06,53,320.168328,950742.699785,4.439022e+06,POINT (951048.965 4439114.857)
2,116.216655,39.996868,945598.585992,4.442241e+06,45,2297.119273,947792.780208,4.442921e+06,POINT (945598.586 4442241.439)


### Candidate Billboard Location
Billboards data were retrieved from LAMAR

In [8]:
sitedf = gpd.read_file("./data/real/haidian_hospital_pro.shp")
sitedf['POINT_X'] = sitedf.geometry.x
sitedf['POINT_Y'] = sitedf.geometry.y
# sites = np.array(sitedf[['NORM_X', 'NORM_Y']], dtype=np.float64)
# print("The number of billboards in Seattle area is ", len(sitedf))
sitedf.head(3)

,经度84,纬度84,POINT_X,POINT_Y,geometry
0,116.257490,40.023362,948914.176201,4.445391e+06,POINT (948914.176 4445390.949)
1,116.287560,39.993906,951677.729470,4.442270e+06,POINT (951677.729 4442270.340)
2,116.353798,39.981356,957422.791546,4.441215e+06,POINT (957422.792 4441214.652)


## Normalization

In [9]:
def Normalization(x, y):
    max_x, max_y = np.max(x), np.max(y)
    min_x, min_y = np.min(x), np.min(y)
    S_x = (max_x-min_x)
    S_y = (max_y-min_y)
    S = max(S_x, S_y)
    new_x, new_y = (x-min_x)/S, (y-min_y)/S
    data_xy = np.vstack((new_x, new_y))
    Data = data_xy.T
    return new_x, new_y, S

In [10]:
ls_X = np.array(ls['POINT_X'])
ls_Y = np.array(ls['POINT_Y'])
bbs_X = np.array(sitedf['POINT_X'])
bbs_Y = np.array(sitedf['POINT_Y'])
X = np.concatenate([ls_X, bbs_X])
Y = np.concatenate([ls_Y, bbs_Y])
NORM_X, NORM_Y, S = Normalization(X, Y)
ls['NORM_X'] = NORM_X[:len(ls)]
ls['NORM_Y'] = NORM_Y[:len(ls)]
sitedf['NORM_X'] = NORM_X[len(ls):]
sitedf['NORM_Y'] = NORM_Y[len(ls):]

# LandScan sites =1109, Billboard sites=839, R=2000 M=20

In [11]:
def gen_real_data(ls, num_sample):
    real_datasets = []
    for i in range(num_sample):
        bbs_ = sitedf
        real_data = {}
        real_data["users"] = torch.tensor(np.array(ls[['NORM_X', 'NORM_Y']])).to(torch.float32)
        real_data["facilities"] = torch.tensor(np.array(bbs_[['NORM_X', 'NORM_Y']])).to(torch.float32)
        real_data["p"] = 15
        real_data["r"] = 2000/S
        real_datasets.append(real_data)
    return bbs_, real_datasets

In [12]:
num_sample = 1
opts.eval_batch_size = 10
opts.max_calc_batch_size = 1280000
width = 1280
bbs_, real_datasets = gen_real_data(ls, num_sample)
from torch.utils.data import DataLoader
from tqdm import tqdm
opts.decode_strategy = 'sampling'
model.eval()
model.set_decode_type(
    "greedy" if opts.decode_strategy in ('bs', 'greedy') else "sampling")
dataloader = DataLoader(real_datasets, batch_size=opts.eval_batch_size)

### Greedy strategy

In [13]:
def get_best(sequences, cost, ids=None, batch_size=None):
    """
    Ids contains [0, 0, 0, 1, 1, 2, ..., n, n, n] if 3 solutions found for 0th instance, 2 for 1st, etc
    :param sequences:
    :param lengths:
    :param ids:
    :return: list with n sequences and list with n lengths of solutions
    """
    if ids is None:
        idx = cost.argmin()
        return sequences[idx:idx+1, ...], cost[idx:idx+1, ...]

    splits = np.hstack([0, np.where(ids[:-1] != ids[1:])[0] + 1])
    mincosts = np.minimum.reduceat(cost, splits)

    group_lengths = np.diff(np.hstack([splits, len(ids)]))
    all_argmin = np.flatnonzero(np.repeat(mincosts, group_lengths) == cost)
    result = np.full(len(group_lengths) if batch_size is None else batch_size, -1, dtype=int)

    result[ids[all_argmin[::-1]]] = all_argmin[::-1]

    return [sequences[i] if i >= 0 else None for i in result], [cost[i] if i >= 0 else math.inf for i in result]

In [14]:
start = time.time()
results = []
for batch in tqdm(dataloader, disable=True):
    batch = move_to(batch, opts.device)
    start = time.time()
    with torch.no_grad():
        if opts.decode_strategy in ('sampling', 'greedy'):
            if opts.decode_strategy == 'greedy':
                assert width == 0, "Do not set width when using greedy"
                assert opts.eval_batch_size <= opts.max_calc_batch_size, \
                    "eval_batch_size should be smaller than calc batch size"
                batch_rep = 1
                iter_rep = 1
            elif width * opts.eval_batch_size > opts.max_calc_batch_size:
                assert opts.eval_batch_size == 1
                assert width % opts.max_calc_batch_size == 0
                batch_rep = opts.max_calc_batch_size
                iter_rep = width // opts.max_calc_batch_size
            else:
                batch_rep = width
                iter_rep = 1
            assert batch_rep > 0
            # This returns (batch_size, iter_rep shape)

            sequences, costs = model.sample_many(batch, batch_rep=batch_rep, iter_rep=iter_rep)
            batch_size = len(costs)
            ids = torch.arange(batch_size, dtype=torch.int64, device=costs.device)
#         else:
#             # assert opts.decode_strategy == 'bs'

#             cum_log_p, sequences, costs, ids, batch_size = model.beam_search(
#                 batch, beam_size=width,
#                 compress_mask=opts.compress_mask,
#                 max_calc_batch_size=opts.max_calc_batch_size
#             )
            if sequences is None:
                sequences = [None] * batch_size
                costs = [math.inf] * batch_size
            else:
                sequences, costs = get_best(
                    sequences.cpu().numpy(), costs.cpu().numpy(),
                    ids.cpu().numpy() if ids is not None else None,
                    batch_size
                )
            duration = time.time() - start
            for seq, cost in zip(sequences, costs):
                seq = seq.tolist()
                results.append((cost, seq, duration))
costs, tours, durations = zip(*results)
print(f"The objective of MCBLP by AM is: {costs[0]}")
end = time.time()-start 
print(f"The running time of DRL is: {end}")

The objective of MCBLP by AM is: 157.9359588623047
The running time of DRL is: 0.45799875259399414


In [15]:
tours

([26, 74, 64, 19, 14, 45, 17, 24, 42, 49, 76, 18, 30, 51, 47],)